<a href="https://colab.research.google.com/github/DaniellePotts/Sentiment-Analysis-with-QReLU-and-mQReLU/blob/main/sentiments_140_QReLU_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install contractions
!python -m spacy download en

     |████████████████████████████████| 284 kB 9.1 MB/s 
     |████████████████████████████████| 321 kB 61.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85445 sha256=118065938fc7af86a0027cdff233afadcf84f8846ad13ff038b04f3773d627e5
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 12.0 MB 8.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
%tensorflow_version 1.x 
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import spacy
import re
import unicodedata
import nltk
nltk.download('stopwords')
import contractions
from collections import Counter
from nltk.tokenize.toktok import ToktokTokenizer

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

from QReLU_TensorFlow_Keras import QReLU
from m_QReLU_TensorFlow_Keras import m_QReLU

TensorFlow 1.x selected.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [5]:
nlp = spacy.load('en', parse=True, tag=True, entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [6]:
cols = ['sentiment','id','date','query_string','user','text']

In [7]:
df = pd.read_csv("./drive/MyDrive/QReLU/training.csv",header=None, names=cols,  engine='python')

In [8]:
sentiments_cat = ['negative','neutral','postive']

In [9]:
def chop_dataframe(df, amount):
  df_4 = df.loc[df['sentiment'] == 4]
  df_1 = df.loc[df['sentiment'] == 0]
  df_final = df_4.head(amount)
  return pd.concat([df_final, df_1.head(amount)])

def encode_corpus(corpus):
  words = ' '.join(corpus).split()
  
  count_words = Counter(words)
  total_words = len(words)

def count_words(corpus):
  words = ' '.join(corpus).split()
  
  counted_words = Counter(words)
  total_words = len(words)

  return total_words, counted_words

def sort_words(corpus):
  total_words, counted_words = count_words(corpus)
  return counted_words.most_common(total_words)

def decode_sentiments(encoded_sentiments, sentiments_cat=sentiments_cat, postive_index=2, 
                      negative_index=0, neutral_index=1):
  decoded_sentiments = []

  for index in range(0, len(encoded_sentiments)):
    if encoded_sentiments[index] == 0:
      decoded_sentiments.append(sentiments_cat[negative_index])
    elif encoded_sentiments[index] == 2:
      decoded_sentiments.append(sentiments_cat[neutral_index])
    elif encoded_sentiments[index] == 4 or encoded_sentiments[index] == 1:
      decoded_sentiments.append(sentiments_cat[postive_index])
  
  return decoded_sentiments

def convert_sentiments(sentiments):
  converted = []

  for s in sentiments:
    if s == 4:
      converted.append(1)
    else:
      converted.append(0)
  return np.array(converted)

In [10]:
#save load json files
import json 

def save(file_name, obj):
  with open(file_name, 'w') as outfile:
    json.dump(obj, outfile)

def load(file_name):
  with open(file_name) as json_file:
    data = json.load(json_file)
    return data['corpus']

In [11]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def convert_to_lower(corpus):
    return [c.lower() for c in corpus]
    
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def expand_contractions(text):
  expanded_words = []    
  for word in text.split():
    # using contractions.fix to expand the shotened words
    expanded_words.append(contractions.fix(word))   
      
  return' '.join(expanded_words)

In [12]:
df_4 = df.loc[df['sentiment'] == 4]
df_1 = df.loc[df['sentiment'] == 0]
df_final = df_4.head(25000)
df_5 = pd.concat([df_final, df_1.head(25000)])

In [13]:
df = df_5

In [ ]:
len(df_5)

50000

In [14]:
sentiments = convert_sentiments(df['sentiment'].values)
corpus = convert_to_lower(df['text'].values)

In [ ]:
corpus[0]

"@switchfoot http://twitpic.com/2y1zl - awww, that's a bummer.  you shoulda got david carr of third day to do it. ;d"

In [15]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        
        # lemmatize text
        # if text_lemmatization:
        #     doc = lemmatize_text(doc)
        
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
            
            pat1 = r'@[A-Za-z0-9_]+'
            pat2 = r'https?://[^ ]+'
            www_pat = r'www.[^ ]+'
            combined_pat = r'|'.join((pat1, pat2))
            doc = re.sub(combined_pat, '', doc)
            doc = re.sub(www_pat, '', doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [16]:
normalized_corpus = normalize_corpus(corpus)
# save("./drive/MyDrive/QReLU/corpus.json", {"corpus": normalized_corpus})

# normalized_corpus = load("./drive/MyDrive/QReLU/corpus.json")

In [17]:
sentences = normalized_corpus

In [18]:
total_words = count_words(normalized_corpus)
sorted_words = sort_words(normalized_corpus)

In [19]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [20]:
def tokenize(corpus):
  reviews_int = []
  for review in corpus:
      r = [vocab_to_int[w] for w in review.split()]
      reviews_int.append(r)
  return reviews_int

In [21]:
encoded = tokenize(normalized_corpus)
encoded_np = np.array(encoded)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
len(encoded)

20000

In [ ]:
#Pre Training Glove embeddings

In [22]:
import numpy as np

embeddings_index = dict()
f = open("/content/drive/MyDrive/QReLU/glove.6B.100d.txt")

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

f.close()
print("Loaded %s word vectors." % len(embeddings_index))

Loaded 400001 word vectors.


In [23]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(normalized_corpus)

In [25]:
vocab_size = len(tokenizer.word_index) + 1
encoded_document = tokenizer.texts_to_sequences(normalized_corpus)

In [26]:
vocab_size

54261

In [27]:
MAXLEN = 160
padded_docs = sequence.pad_sequences(encoded_document, MAXLEN, padding='post')

In [28]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [29]:
# VOCAB_SIZE = total_words[0]
MAXLEN = 160 #the max length of a tweet

X_train, X_test, y_train, y_test = train_test_split(
    encoded, sentiments, test_size=0.2, random_state=42)

X_train = sequence.pad_sequences(X_train, MAXLEN)
X_test = sequence.pad_sequences(X_test, MAXLEN)

In [31]:
len(embedding_matrix), embedding_matrix.size, MAXLEN, vocab_size

(54261, 5426100, 160, 54261)

In [42]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=MAXLEN),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1),
    QReLU()
])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 100)          5426100   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 160, 32)           9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 80, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               49664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
q_re_lu_1 (QReLU)            (None, 1)                

In [43]:
model2.compile(loss="binary_crossentropy",optimizer="adam",metrics=['acc'])

history = model2.fit(X_train, y_train, epochs=6, validation_split=0.2, batch_size=32, shuffle=True)

Train on 32000 samples, validate on 8000 samples
Epoch 1/6
32000/32000 [==============================] - 219s 7ms/sample - loss: 0.7561 - acc: 0.5268 - val_loss: 0.6508 - val_acc: 0.6346
Epoch 2/6
32000/32000 [==============================] - 215s 7ms/sample - loss: 0.6177 - acc: 0.6881 - val_loss: 0.6106 - val_acc: 0.6570
Epoch 3/6
32000/32000 [==============================] - 216s 7ms/sample - loss: 0.5548 - acc: 0.7523 - val_loss: 0.5692 - val_acc: 0.7107
Epoch 4/6
32000/32000 [==============================] - 216s 7ms/sample - loss: 0.4489 - acc: 0.8159 - val_loss: 0.7942 - val_acc: 0.7322
Epoch 5/6
32000/32000 [==============================] - 218s 7ms/sample - loss: 0.3941 - acc: 0.8543 - val_loss: 0.8156 - val_acc: 0.7191
Epoch 6/6
32000/32000 [==============================] - 216s 7ms/sample - loss: 0.2646 - acc: 0.9205 - val_loss: 1.0998 - val_acc: 0.7322


In [44]:
results = model2.evaluate(X_test, y_test)
print(results)

10000/10000 [==============================] - 14s 1ms/sample - loss: 1.1267 - acc: 0.7238
[1.1267444622039795, 0.7238]


In [36]:
X_test[0], y_test[0]

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [39]:
model2.predict(X_test[0:2])

array([[0.42516795],
       [0.546085  ]], dtype=float32)